# Lab 7: Introduction to Markov chains and random graphs

In this lab we will be looking at Markov Processes.  Below is the code to import the modules we will be using throughout this lab.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import scipy as sp
import scipy.stats as st
import pickle as pkl
import csv as csv
import networkx as nx  # If using your own computer may need to install networkx 
print ("Modules Imported!")

Modules Imported!


## Introduction to Markov Chains:

A Markov chain $(X[t]: t\geq 0)$ is a sequence of random variables such that the distribution of each variable is determined by the random variable before it.  Equivalently, it is a sequence of states visited by a form of finite state machine with random transitions.  To describe how to generate the chain we specify the initial probability distribution $\pi_0$ and a one-step transition probability matrix $P$.   We assume there is a finite state space described by {0,1,, . . . , $n-1$} for some $n$.  The intial distribution, $\pi(0)$, should be a probability vector (nonnegative entries with sum equal to one) of length $n$.   It consists of the probability the process is in a given state at time zero: ${\pi_i(0)} = P\{X[0]=i\}$ for $0\leq i \leq n-1.$    The matrix $P$ is an $n\times n$ matrix such that each row of the matrix is a probability vector.   If the state of the Markov chain at time $t-1$ is $i$ (in other words, if $X[t-1]=i$), then the (conditional) probability distribution of $X[t]$, the state at time $t$, is given by the ith row of $P.$ Additionally, we can look at the individual entries: $p_{ij}=P\left\{X[t]=j \; \big| \; X[t-1]=i\right\}$.  Below is a class that can be used to create a Markov chain.  An object in this class is created with specified values of $\pi_0$ and $P$, and can be used to calculate the state distribution at any time or to generate a sample path (i.e. a variate of the sequence of states) of the Markov chain.  

The state distributions are determined recursively as follows.  If $$\pi_i(t)=P\{X[t]=i\}$$ then by the law of total probability, $$\pi_j(t)= P\{X[t]=j\}=\sum_i P\{X[t-1]=i,X[t]=j\} =\sum_i \pi_i(t-1)p_{i,j}$$  That is, if we view $\pi(t)$ as a row vector for all $t,$ then $\pi(t)=\pi(t-1)P,$  where we have used multiplication of a row vector by a matrix.  (A nice introduction to Markov chains is given on wikipedia.)

In [2]:
# Markov chain object similar to random variable objects in scipy.stats (st)

class Markov_chain:
    def __init__(self, P, pi0):   # The transition probability matrix and initial distribution are passed
        self.P = P 
        self.pi0 = pi0
        self.n=np.size(pi0)
        if np.shape(P) != (self.n,self.n):
            print ("Error: Shape of P doesn't match pi0")
   
    def rvs(self,T):     # produce a length T segment of variates of the random process
        X = [np.random.choice(self.n, p=self.pi0)] 
        for t in range(1,T):
            X.append(np.random.choice(self.n, p=self.P[X[t-1],:]))
        return np.array(X)
    
    def pi(self,t):    # produce probability distribution at time t
        pi_new=self.pi0
        for s in range(t):
            pi_new = np.dot(pi_new, P)    # one step update of probability distribution
        return pi_new

# An example of how you can use the class to create and run through a simple two state chain
P=np.array([[0.9,0.1],[0.1,0.9]])
pi0=np.array([0.1,0.9])
print ('Simulated state sequence')
markov=Markov_chain(P,pi0)  
print (markov.rvs(100))
for t in range(20):
    print ("Distribution at time", t, ":", markov.pi(t))

Simulated state sequence
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Distribution at time 0 : [0.1 0.9]
Distribution at time 1 : [0.18 0.82]
Distribution at time 2 : [0.244 0.756]
Distribution at time 3 : [0.2952 0.7048]
Distribution at time 4 : [0.33616 0.66384]
Distribution at time 5 : [0.368928 0.631072]
Distribution at time 6 : [0.3951424 0.6048576]
Distribution at time 7 : [0.41611392 0.58388608]
Distribution at time 8 : [0.43289114 0.56710886]
Distribution at time 9 : [0.44631291 0.55368709]
Distribution at time 10 : [0.45705033 0.54294967]
Distribution at time 11 : [0.46564026 0.53435974]
Distribution at time 12 : [0.47251221 0.52748779]
Distribution at time 13 : [0.47800977 0.52199023]
Distribution at time 14 : [0.48240781 0.51759219]
Distribution at time 15 : [0.48592625 0.51407375]
Distribution at time 16 : [0.488741 0

We won't go into this deeply, but notice how as the distributions change in time, they appear to be converging. For some Markov chains, there exists a limit for the distribution. If this occurs then the limit distribution $\pi$ satisfies $\pi = \pi P$ and $\pi$ is called an *equilibrium distribution.*

<br>**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 1:</SPAN>** Imagine that the states {0, . . . , n-1} of a particular Markov chain are the nodes of a ring graph, with state 0 following state n-1.   Create a Markov_chain object for n=6 with inital state 0 (so x[0]=0 with probability one) with the following transition probabilities:  given X[t-1]=i, X[t] is equal to i with probability 0.8 and to i+1 (modulo n) with probability 0.2.
<ol>
<li> Print the first 100 states for a simulation of the chain.</li>
<li> Simulate the chain for 10,000 time units, and calculate the number of complete revolutions the chain makes around the ring, divided by 10,000.  This is an estimate of the long term cycle rate for the Markov chain.  Repeat the experiment ten times to get an idea of the variation from one experiment to the next.</li>
<li> Give an analytical argument for what the long term cycle rate for this Makov chain should be.</li>

In [3]:
# Your code here

class Markov_Ring_Chain:
    #P: transfer probability
    #S_init: initial state
    #n: number of total state
    def __init__(self, P, S_init, n):   # The transition probability matrix and initial distribution are passed
        self.P = P 
        self.S_init = S_init
        self.n = n
        
    # transfer function for Makov chain
    def transfer(self, cur_state):
        next_state = cur_state
        # 0.8 probability for remaining the current state
        # 0.2 probability for 
        if np.random.choice(2,p=self.P):
            next_state = (next_state+1) % self.n
        return next_state
    
    # run T times, retun the simulation array
    def rvs (self, T):
        x = [self.S_init]
        for _ in range (T):
            x.append(self.transfer(x[-1]))
        return np.array(x)
    
    # run T times, return the cycle number
    def get_cycle_times(self, T):
        x = [self.S_init]
        cycle = 0
        for _ in range (T):
            x.append(self.transfer(x[-1]))
            if x[-1] == self.n -1 and not(x[-1]==x[-2]):
                cycle += 1                  # increase count if run the whole cycle
        return cycle
############################ Question 1 ############################
n = 6               # state number
S_init = 0          # initial state is 0
P = [0.8,0.2]       # transfer probability list
Markov = Markov_Ring_Chain(P,S_init,n)

print ('first 100 states for a simulation of the chain:\n', Markov.rvs(100))
############################ Question 2 ############################
T = 10000
m = Markov.get_cycle_times(T)
print ('complete revolutions the chain makes around the ring after 10000 states:\n', m)
print ('cycle rate in the simulation:\n', m/T)

first 100 states for a simulation of the chain:
 [0 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 5 0 0
 0 0 0 0 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 4 4 4 4 4 4 4
 5 5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 3]
complete revolutions the chain makes around the ring after 10000 states:
 324
cycle rate in the simulation:
 0.0324


__Answer:__ (Your answer here)

<ol>
<li> Print the first 100 states for a simulation of the chain.</li>
    <ol>
        The first 100 states for a simulation of the chain has been shown above.
    </ol>
<li> Simulate the chain for 10,000 time units, and calculate the number of complete revolutions the chain makes around the ring, divided by 10,000.  This is an estimate of the long term cycle rate for the Markov chain.  Repeat the experiment ten times to get an idea of the variation from one experiment to the next.</li>
     <ol>
       <li>Result has been shown above.Complete revolutions the chain makes around the ring after 10000 states is about 333.</li>
       <li>The cycle rate in the simulation is about 0.0333</li>
    </ol>
<li> Give an analytical argument for what the long term cycle rate for this Makov chain should be.</li>
    <ol>
        The ring length is 6, and the probability to transfer to the next state is 0.2. Therefore, the theoretical cycle rate should be transfer probability / ring length, that is 0.2/6 = 00.333.
    </ol>
</ol>

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 1</SPAN>**

## Markov Gibberish

Let's look at some applications of a Markov Chain. When I was younger I remember using AIM (instant messenger) to talk to my friends. At some point people started to create bots which could hold conversations with users. Of course we had fun trying to get them to say things that didn't make sense, but the idea that it could generate somewhat relevant conversations was pretty cool. We're going to build a rudimentary version of that here. The code below generates text based on a large sample text, such that each word is generated based on the two words just before it.  The generated sequence of words is given by a 2-Markov chain (here "2" means that the distribution of the value of each word is determined by the two previous words, instead of just the one previous word).  That means that any three consecutive words in the generated text are also consecutive words somewhere in the sample text.   If words w1 w2 were the last two words added to the generated text string, then to generate the next word we look for all occurrences of w1 w2 as consecutive words in the sample text, and see what word was just after them for each occurrence.   The randomly generated text selects the next word to match one of those occurrences, selected at random with equal probability.

In [4]:
# Based on http://agiliq.com/blog/2009/06/generating-pseudo-random-text-with-markov-chains-u/    --BH
# Next word of random string is selected from possible word that follow the last two words.


class Markov_text:
    def __init__(self, open_file):   # The single parameter passed is a file handle
        self.cache = {}  # Will be dictionary mapping a key (two consecutive words) to possible next word
        self.open_file = open_file
        self.words = self.file_to_words()  # Read the words from the file into array self.words
        self.word_size = len(self.words)
        self.database()   # Fill in the dictionary
       
       
    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words
           
       
    def triples(self):
    #Generates triples from the given data string. So if our string were
    # "What a lovely day", we'd generate (What, a, lovely) and then (a, lovely, day)."""
           
        if self.word_size < 3:
            return
         
        for i in range(self.word_size - 2):
            yield (self.words[i], self.words[i+1], self.words[i+2])    #Like return but returns a generator to be used once
               
    def database(self):
        for w1, w2, w3 in self.triples():
            key = (w1, w2)
            if key in self.cache:
                self.cache[key].append(w3)
            else:
                self.cache[key] = [w3]
                         
    def generate_markov_text(self, size=100):
        seed = np.random.randint(0, self.word_size-3)
        w1, w2 = self.words[seed], self.words[seed+1]  # Initial key is (w1,w2)
        gen_words = [w1,w2]
        for i in range(size):
            w1, w2 = w2, np.random.choice(self.cache[(w1, w2)])
            gen_words.append(w2)
        return ' '.join(gen_words)
                
        
        
file_ = open('Jeeves.txt')
markov = Markov_text(file_)   # Creates the object markov using the file
markov.generate_markov_text()   # Generate random text

# The above has a slight bug.   It is possible for the key to be the
# last two words in the text file and have no entry in the dictionary.
# This won't happen if the last two words in the text file appear
# consecutively somewhere else in the file.   -BH

'done half a mile of him." Jimmy frowned. "That\'s awkward," he said. "I\'ll tell you now, but often in this astounding city! New York from itself; to force it--in his picturesque phrase--to hit the trail. This time I\'m convinced that I nearly swooned. Jeeves was what American chappies would call again later, sir." "Something must be whole squads of the year he rested. I didn\'t care a hang for; yet here was I now, dripping doubloons and pieces of eight and longing to hand over your money to won\'t let you, whereas the chappies you care for a quiet cigarette one morning'

With punctuation and everything, this initially looks like a bit of a mess. However, the sentences themselves are coming together and can make a little bit of sense. It is a very primitive version of those bots on instant messaging.

<br>**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 2:</SPAN>** Using the class above, do the following:
<ol><li> Find your own text file or copy and paste from somewhere and use the text to create gibberish from that. To work well, your file really needs to contain a large amount of text. The "Jeeves" file was pulled from a short story. Print out your own 100 word gibberish paragraph based on the text file you used.</li>
<li> Rewrite the code to create a 1-Markov Chain of the words instead of the 2-Markov Chain given above. Print out your own 100 word gibberish paragraph from this.</li>
<li> Rewrite the code to create a 3-Markov Chain of the words instead of the 2-Markov Chain given above. Print out your own 100 word gibberish paragraph from this.</li>
<li> Comment on the qualitative differences of your paragraphs using these methods.</li>

In [5]:
# Your code here (add additional code cells if necessary)
############################ Question 1 ############################
file_ = open('The Count of Monte Cristo_my version.txt')
markov = Markov_text(file_)   # Creates the object markov using the file
print("2-Markov :\n",markov.generate_markov_text())   # Generate random text

########################### Question 2 ############################
class Markov1_text:
    def __init__(self, open_file):   # The single parameter passed is a file handle
        self.cache = {}  # Will be dictionary mapping a key (two consecutive words) to possible next word
        self.open_file = open_file
        self.words = self.file_to_words()  # Read the words from the file into array self.words
        self.word_size = len(self.words)
        self.database()   # Fill in the dictionary
       
       
    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words
           
       
    def doubles(self):
    #Generates triples from the given data string. So if our string were
    # "What a lovely day", we'd generate (What, a, lovely) and then (a, lovely, day)."""
           
        if self.word_size < 2:
            return
         
        for i in range(self.word_size - 1):
            yield (self.words[i], self.words[i+1])    #Like return but returns a generator to be used once
               
    def database(self):
        for w1, w2 in self.doubles():
            key = w1
            if key in self.cache:
                self.cache[key].append(w2)
            else:
                self.cache[key] = [w2]
                         
    def generate_markov_text(self, size=100):
        seed = np.random.randint(0, self.word_size-2)
        w1 = self.words[seed]  # Initial key is (w1,w2)
        gen_words = [w1]
        for i in range(size):
            w1 = np.random.choice(self.cache[w1])
            gen_words.append(w1)
        return ' '.join(gen_words)

markov1 = Markov1_text(file_)   # Creates the object markov using the file
print("1-Markov :\n",markov1.generate_markov_text())   # Generate random text

########################### Question 3 ############################
class Markov3_text:
    def __init__(self, open_file):   # The single parameter passed is a file handle
        self.cache = {}  # Will be dictionary mapping a key (two consecutive words) to possible next word
        self.open_file = open_file
        self.words = self.file_to_words()  # Read the words from the file into array self.words
        self.word_size = len(self.words)
        self.database()   # Fill in the dictionary
       
       
    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words
           
       
    def four(self):
    #Generates triples from the given data string. So if our string were
    # "What a lovely day", we'd generate (What, a, lovely) and then (a, lovely, day)."""
           
        if self.word_size < 4:
            return
         
        for i in range(self.word_size - 3):
            yield (self.words[i], self.words[i+1],self.words[i+2],self.words[i+3])    #Like return but returns a generator to be used once
               
    def database(self):
        for w1, w2, w3, w4 in self.four():
            key = (w1,w2,w3)
            if key in self.cache:
                self.cache[key].append(w4)
            else:
                self.cache[key] = [w4]
                         
    def generate_markov_text(self, size=100):
        seed = np.random.randint(0, self.word_size-4)
        w1,w2,w3 = self.words[seed], self.words[seed+1], self.words[seed+2]  # Initial key is (w1,w2)
        gen_words = [(w1,w2,w3)]
        for i in range(size):
            w1, w2, w3 = w2, w3, np.random.choice(self.cache[(w1.w2.w3)])
            gen_words.append(w3)
        return ' '.join(gen_words)

markov3 = Markov3_text(file_)   # Creates the object markov using the file
print("3-Markov generator:\n",markov1.generate_markov_text())   # Generate random text


2-Markov :
 "it seems to me that Imust ask your leave of oldDantes, he went downstairs to rejoin Danglars, who awaitedhim at the excessive honor paid to hisson. "And why did you refuse, my son?" inquired the old man's strength failed him, and then, drawing Dantes on oneside, he said -- "Let go!" The anchor was instantly dropped, and the oldmust make way for the profits ofthe voyage." Then, as they were just passing the Round Tower, the youngman shouted: "Stand by there to lower the topsails and jib;brail up the spanker!" The order was executed as promptly as it seems to me." "So,
1-Markov :
 now that is aneighbor who awaitedhim at Porto-Ferrajo?" "Yes, he had gone on the two oarsmen bent to do not want foranything; and it isnot pleasant to the question,sir. Do you all mortal, and it appears that you know, you require, Dantes. "No doubt, my father is true!" cried Dantes, his care." "Of what I thankyou in your wine?" said Dantes, who could judge from their work, and touchat Elba, althoug

__Answer:__ (Your answer here)
<li>Comment on the qualitative differences of your paragraphs using these methods.</li>
The sentence generated by 3-Markov makes more sense than the sentence generateed by 2-Markov, and 2-Markov is better than 1-Markov. In addition, the sentence generated by 3-Markov is more close to the original txt while 1-Markov is more mutable.

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 2</SPAN>**

## Random Graphs and Working with Graphs

Markov chains that are time independent and finite can be represented really well by graphs. I'm sure you've used finite state machines to describe processes or sequences in some of your other classes. They can also be useful in analyzing or describing networks. Networkx is an extensive Python package for generating and analyzing graphs (the kind with nodes and edges) and can also plot them.  The following code adapted from the networkx webpage generates a random undirected graph by placing nodes at random locations within the unit square and connecting nodes together if their distance is less than a specified threshold.

In [6]:
#import networkx as nx  DEBUGGING

d=0.16 # distance threshold, pairs of nodes within distance d are connected by an edge
G=nx.random_geometric_graph(100,d)  #100 nodes in unit square, distance threshold d
# position is stored as node attribute data for random_geometric_graph
pos=nx.get_node_attributes(G,'pos')  # pos is a dictionary, with the indices as keys, ranging from 0 to 99

# find node nearest to (0.5,0.5), called the center node
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

print ('The center node is located at:', pos[ncenter][0], ',', pos[ncenter][1])
# color by path length from node near center
p=nx.single_source_shortest_path_length(G,ncenter)  #p is dictionary: keys are nodes, values are distances to center
                                                    #Node not connected to the center node are not listed in p.
                                                    # p include the center node itself.
if nx.is_connected(G):   # The Networkx method is_connected checks for connectness
    print ("The graph is connected")
else: print ("The graph is not connected")

plt.figure(figsize=(8,8))
# nx.draw_networkx_edges(G,pos,nodelist=[ncenter],alpha=0.4)  #All edges are drwn
nx.draw_networkx_edges(G,pos,alpha=0.4)  #All edges are drwn
nx.draw_networkx_nodes(G,pos,alpha=0.3,node_size=80)  #All nodes are drawn but nearly transparent
nx.draw_networkx_nodes(G,pos,nodelist=p.keys(),   # Nodes connected to center node are drawn, closer nodes have darker color
                       node_size=80)

plt.xlim(-0.05,1.05)
plt.ylim(-0.05,1.05)
plt.axis('off')
plt.savefig('random_geometric_graph.png')
plt.show()

The center node is located at: 0.4869168305108754 , 0.4869341858203452
The graph is connected


Excellent documentation for NetworkX is available online (search for NetworkX documentation). We find a plethora of classes for generating both undirected graphs (no arrows on the edges) and directed graphs.   Run the code in the above cell a few dozen times, and see how often the graph generated is *connected*, meaning that there is a path through the graph from any node to any other node.   One way a graph could be not connnected is to have an isolated node, which is a node with no neighbors.   Also, try changing the value of the threshold d and seeing the effect on connectivity.

<br>**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 3:</SPAN>**   Estimate the probability of connectivity for the above model (100 nodes at random locations in unit square with edges between pairs of nodes at distance d or smaller) for values of d in 0.10, 0.11, . . . , 0.20 by generating 1000 instances for each d and seeing the fraction of them that are connected. Is there a sort of threshold where the odds of a connected graph jumps significantly?

In [7]:
# Your code here
def Random_graph(d,T):
    count = 0
    for _ in range (T):
        G=nx.random_geometric_graph(100,d)  #100 nodes in unit square, distance threshold d
        dmin=1
        ncenter=0
        pos=nx.get_node_attributes(G,'pos')  # pos is a dictionary, with the indices as keys, ranging from 0 to 99
        for n in pos:
            x,y=pos[n]
            dis=(x-0.5)**2+(y-0.5)**2
            if dis<dmin:
                ncenter=n
                dmin=dis
        p=nx.single_source_shortest_path_length(G,ncenter)
        if nx.is_connected(G):   # The Networkx method is_connected checks for connectness
            count += 1
    return count

for i in np.linspace(0.10,0.20,11):
    print("With d = %2f, the fraction of connected: "%i,Random_graph(i,1000))
    

With d = 0.100000, the fraction of connected:  0
With d = 0.110000, the fraction of connected:  0
With d = 0.120000, the fraction of connected:  0
With d = 0.130000, the fraction of connected:  19
With d = 0.140000, the fraction of connected:  103
With d = 0.150000, the fraction of connected:  285
With d = 0.160000, the fraction of connected:  523
With d = 0.170000, the fraction of connected:  661
With d = 0.180000, the fraction of connected:  826
With d = 0.190000, the fraction of connected:  875
With d = 0.200000, the fraction of connected:  921


__Answer:__ (Your answer here)

The fraction of connected has been shown above. As we can see, when the distance threshold > 0.14, the fraction jumps significantly.

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 3</SPAN>**

The probability of connectivity can be approximately computed analytically as follows.  You might have noticed in the simulations that the main cause of disconnectivity is having at least one isolated node (no neighbors).

<br>**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 4:</SPAN>**   Consider the same model.  Imagine that 100 nodes, numbered from 1 to 100, are independently placed in the unit square with a uniform distribution for each node, and then edges are placed between pairs of nodes at distance d or smaller.  Except for the second half of the fourth part below, your answers should be an expression involving $d.$   No computation is required.
<ol><li> Focus on node 1. Suppose for simplicity it is not near the boundary of the square, so the entire disk of radius $d$ around the node is in the square.   What is the probability that a given other node (say, node 2) is *not* within distance $d$ of node 1?
<li>  What is the probability node 1 is isolated (i.e. *none* of the other nodes is within distance $d$ of node 1)?
<li>  What is the expected number of isolated nodes?
<li>  Assume (this is a reasonable approximation) the total number of isolated nodes has the Poisson distribution with mean given by the answer to the previous problem part.   What is the probability that there are no isolated nodes?
Compute the numerical values for  d in 0.10, 0.11, . . . , 0.20 and compare to the estimated probability the
graph is connected in Problem 3. </ol>

__Answer:__ (Your answer here)
<li>The probability of not within distance d of node 1 is equal to the remaining area devide by the total area. 
    That is $$P = 1-\pi d^2$$</li>
<li>If the node 1 is isolated, it means that the other 99 nodes are not in the region of node 1. Therefore, we can use the probability we get from part 1. Here we denote the probability of node 1 is isolated as $P_{1i}$</li> $$P_{1i} = P^{99} = (1-\pi d^2)^{99}$$

<li>We time the probability of single node that is isolated by total number of nodes $n=100$. Here we denote the expected number of isolated nodes as $E[X]$ $$E[X]=100P_{1i} =100(1-\pi d^2)^{99}$$</li>
<li>Use the poisson distribution, we get:$$P(k=0)=e^{-100(1-\pi d^2)^{99}}$$, the values are shown below in code result. The probability there we calculate from theoratical analysis is larger than the probability in Problem 3. The reason is that here we assume all the region disks with radius $d$ are entirely insded our $1\times 1$ plane. However, in real case there will be some nodes are near the boundary, which makes the result probability different. That's why our result will be a slightly larger.</li>

In [8]:
# Your code here (if necessary)
from math import pi,e
for d in np.linspace(0.1,0.2,11):
    lamb = 100*(1-pi*d**2)**99
    p = e**(-lamb)
    print ("distance = %f, the expaction of isolated nodes: "%d,p)

distance = 0.100000, the expaction of isolated nodes:  0.014375382556273155
distance = 0.110000, the expaction of isolated nodes:  0.11573548129613279
distance = 0.120000, the expaction of isolated nodes:  0.3597690669272207
distance = 0.130000, the expaction of isolated nodes:  0.6368236824267614
distance = 0.140000, the expaction of isolated nodes:  0.8309712238173423
distance = 0.150000, the expaction of isolated nodes:  0.9319422565913525
distance = 0.160000, the expaction of isolated nodes:  0.9754666715905875
distance = 0.170000, the expaction of isolated nodes:  0.9919479603219504
distance = 0.180000, the expaction of isolated nodes:  0.9975789752005879
distance = 0.190000, the expaction of isolated nodes:  0.999332667029149
distance = 0.200000, the expaction of isolated nodes:  0.9998316800547294


**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 4</SPAN>**

## Lab Questions:

Make sure to complete all lab questions 1-4 for this weeks lab.

<div class="alert alert-block alert-warning"> 
## Academic Integrity Statement ##

By submitting the lab with this statement, you declare you have written up the lab entirely by yourself, including both code and markdown cells. You also agree that you should not share your code with anyone else. Any violation of the academic integrity requirement may cause an academic integrity report to be filed that could go into your student record. See <a href="https://provost.illinois.edu/policies/policies/academic-integrity/students-quick-reference-guide-to-academic-integrity/">Students' Quick Reference Guide to Academic Integrity</a> for more information. 